In [ ]:
import holoviews as hv
hv.extension('bokeh')
import pandas as pd
import numpy as np
import xarray as xr
from holoviews import opts, dim

print(hv.__version__)

Holoviews accepts several different input data formats to construct Elements. Generally, as long as one can define `kdim` and `vdims` consistently, holoviews is quite smart about handling data.

Options include, but are not limited to:
1. Tuples
1. numpy arrays
1. pandas dataframes
1. xarray datasets/dataarrays

1.-3. fall in the category [Tabular Datasets](http://holoviews.org/user_guide/Tabular_Datasets.html), while the 4th. is somewhat special in that it is a [Gridded Dataset](http://holoviews.org/user_guide/Gridded_Datasets.html) which makes asssumptions about the dimensions.

Let's start in the back and consider the most practical use cases first: pandas dataframes and xarray datasets. This is because 1. and 2. make it harder to keep track of associated metadata (such as variable names).

Dataframes are tabular, and so we can choose any column we like to be key or value dimensions:

In [ ]:
df = pd.DataFrame({
    'Time': np.tile(np.arange(100), 9),
    'y': np.random.randn(900).cumsum(axis=0),
    'gridsize': np.repeat([0, 1, 2, 0, 1, 2, 0, 1, 2], 100),
    'turbulence': np.repeat([0, 0, 0, 1, 1, 1, 2, 2, 2], 100)})

hv.Scatter(df, kdims='Time', vdims='gridsize') + hv.Scatter(df, kdims='turbulence', vdims='y')

Xarray datasets, on the other hand, have an inherent understanding about what are key dimensions (the coordinates) and what are value dimensions.

In [ ]:
ds = xr.DataArray(np.random.rand(5,7), 
                  coords={'x': (['x'], np.arange(5)), 'y': (['y'], np.arange(7))},
                  dims=['x', 'y']
                ).rename('z').to_dataset()

In [ ]:
hv.Points(ds).opts(opts.Points(color=hv.dim('z'), colorbar=True, size=2))

What happened is that holoviews automatically understood that x and y are `kdims` and z is a `vdim`. Cf.:

In [ ]:
#hv.Points(ds, kdims=['x', 'z'], vdims='y') # does not work!

Which yields a 
```
DataError: xarray Dataset must define coordinates for all defined kdims, [Dimension('z')] coordinates not found.

XArrayInterface expects gridded data, for more information on supported datatypes see http://holoviews.org/user_guide/Gridded_Datasets.html
```

Now for the first two options:
1.: Not the recommended way, IIRC, but works. 
2.: works as well.

In [ ]:
data_tuple = (np.arange(10), 10*np.random.rand(10))
data_tuple2 = ([chr(65+i) for i in range(10)], list(range(10)))
data_array = np.random.rand(10,2)

hv.Scatter(data_tuple, kdims='x', vdims='y') + hv.Points(data_array) + hv.Bars(data_tuple2)

When in doubt, one can always try and construct a `Dataset` and see what happens by default:

In [ ]:
hv.Dataset(ds)

In [ ]:
hv.Dataset(df)